# Modelos de prediccion
Vamos a probar distintos enfoques:
Enfoques de serie de tiempo:
* Suavizado exponencial #investigar

Enfoques Machine Learning:
* XGboost: Emiii
* LIGHT_GBM: Rugo
* Catboost: Vano

In [ ]:
from DataLoader import importar_ventas

In [ ]:

FEATURES=['day','month','year', 'demand', 'mean_price', 'cluster', 'cod_store_subgroup',
       'demand_lag_1', 'demand_lag_2', 'demand_lag_3', 'demand_lag_4',
       'demand_lag_5', 'demand_lag_6', 'demand_lag_7']
TARGET=['demand']

test=ventas.groupby("cod_store_subgroup").tail(7)
train=ventas.drop(test.index)

X_train=train[FEATURES]
y_train=train[TARGET]
X_test=test[FEATURES]
y_test=test[TARGET]